# Imports

In [1]:
#Basic imports
import numpy as np
import pandas as pd
from sklearn import preprocessing

#sklearn imports
from sklearn.decomposition import PCA #Principal Component Analysis
from sklearn.manifold import TSNE #T-Distributed Stochastic Neighbor Embedding
from sklearn.cluster import KMeans #K-Means Clustering
from sklearn.preprocessing import StandardScaler #used for 'Feature Scaling': failed

from sklearn.preprocessing import MinMaxScaler # used for feature rescaling : failed

from sklearn.preprocessing import Normalizer # used for normalizing data : failed

#plotly imports
import plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

ImportError: cannot import name 'add_newdocs'

# Read the data

In [2]:
#df is our original DataFrame of Tuesday
df = pd.read_csv("K:/CIC-2017-dataset/CIC-IDS-2017/MachineLearningCVE/Tuesday-WorkingHours.pcap_ISCX.csv")
print(df.shape)        # (445909, 79)

H:\anacondaa\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning:

Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.



(445909, 79)


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 445909 entries, 0 to 445908
Data columns (total 79 columns):
 Destination Port               445909 non-null int64
 Flow Duration                  445909 non-null int64
 Total Fwd Packets              445909 non-null int64
 Total Backward Packets         445909 non-null int64
Total Length of Fwd Packets     445909 non-null int64
 Total Length of Bwd Packets    445909 non-null int64
 Fwd Packet Length Max          445909 non-null int64
 Fwd Packet Length Min          445909 non-null int64
 Fwd Packet Length Mean         445909 non-null float64
 Fwd Packet Length Std          445909 non-null float64
Bwd Packet Length Max           445909 non-null int64
 Bwd Packet Length Min          445909 non-null int64
 Bwd Packet Length Mean         445909 non-null float64
 Bwd Packet Length Std          445909 non-null float64
Flow Bytes/s                    445708 non-null object
 Flow Packets/s                 445909 non-null object
 Flow IAT Mean 

# Exploration/ Engineering

First, we construct a new DataFrame, X that we can modify. X will begin as a 'copy' of the original DataFrame, df

In [4]:
X = df.copy()

Test if there any missing values in DataFrame. It turns out there are missing values

In [5]:
X.isnull().sum()

 Destination Port                 0
 Flow Duration                    0
 Total Fwd Packets                0
 Total Backward Packets           0
Total Length of Fwd Packets       0
 Total Length of Bwd Packets      0
 Fwd Packet Length Max            0
 Fwd Packet Length Min            0
 Fwd Packet Length Mean           0
 Fwd Packet Length Std            0
Bwd Packet Length Max             0
 Bwd Packet Length Min            0
 Bwd Packet Length Mean           0
 Bwd Packet Length Std            0
Flow Bytes/s                    201
 Flow Packets/s                   0
 Flow IAT Mean                    0
 Flow IAT Std                     0
 Flow IAT Max                     0
 Flow IAT Min                     0
Fwd IAT Total                     0
 Fwd IAT Mean                     0
 Fwd IAT Std                      0
 Fwd IAT Max                      0
 Fwd IAT Min                      0
Bwd IAT Total                     0
 Bwd IAT Mean                     0
 Bwd IAT Std                

In [6]:
X.isnull().values.any()

True

In [7]:
X.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,88,640,7,4,440,358,220,0,62.857143,107.349008,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,88,900,9,4,600,2944,300,0,66.666667,132.287566,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,88,1205,7,4,2776,2830,1388,0,396.571429,677.274651,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,88,511,7,4,452,370,226,0,64.571429,110.276708,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,88,773,9,4,612,2944,306,0,68.000000,134.933317,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [8]:
listLabel = list(X[" Label"].drop_duplicates().values)
print('\n The types of all traffics are {}'.format(listLabel))


 The types of all traffics are ['BENIGN', 'FTP-Patator', 'SSH-Patator']


In [9]:
X[" Label"].unique()

array(['BENIGN', 'FTP-Patator', 'SSH-Patator'], dtype=object)

In [10]:
benignTr = X.loc[X[" Label"] == 'BENIGN']
ftpTr = X.loc[X[" Label"] == 'FTP-Patator']
sshTr = X.loc[X[" Label"] == 'SSH-Patator']


print ('\n The number of Benign traffics = {}'.format(len(benignTr))) 

print ('\n The number of FTP-Patator traffics = {}'.\
       format(len(ftpTr))) # 
print ('\n The number of SSH-Patator traffics = {}'.\
       format(len(sshTr))) # 



 The number of Benign traffics = 432074

 The number of FTP-Patator traffics = 7938

 The number of SSH-Patator traffics = 5897


# Data Cleaning

In [11]:
X = df.drop(" Label", axis=1)    #(445909, 78)
y = df[" Label"]                #(445909,)

# Clustering

Now, before we get into clustering our data, we just need to do one more thing: feature-scale our numerical variables.

We need to do this because, while each of our categorical variables hold values of either 0 or 1, some of our numerical variables hold values like 2596 and 2785. If we were to leave our data like this, then K-Means Clustering would not give us such a nice result, since K-Means Clustering measures the euclidean distance between data-points. This means that, if we were to leave our numeical variables un-scaled, then most of the distance measured between points would be attributed to the larger numerical variables, rather than any of the categorical variables.

To fix this problem we will scale all of our numerical variables through the use of sklearn's StandardScaler tool. This tool allows us to scale each numerical variable such that each numerical variable's mean becomes 0, and it's variance becomes 1. This is a good way to make sure that all of the numerical variables are on roughly the same scale that the categorical (binary) variables are on.

But, to make sure we scale only our numerical variables -- and not our categorical variables --, we'll split our current DataFrame, X, into two other DataFrames: numer and cater; feature-scale. numer, then recombine the two DataFrames together again into a DataFrame that is suitable for clustering.

In [12]:
#numer is the DataFrame that holds all of X's numerical variables
numer = X[[" Destination Port",
           " Flow Duration",
           " Total Fwd Packets",
           " Total Backward Packets",
           "Total Length of Fwd Packets",
           " Total Length of Bwd Packets",
           " Down/Up Ratio",
           " Fwd Packet Length Max",
           " Fwd Packet Length Min",
           " Fwd Packet Length Mean",
           " Fwd Packet Length Std",
           "Bwd Packet Length Max",
           " Bwd Packet Length Min",
           " Bwd Packet Length Mean",
           " Bwd Packet Length Std",
           "Flow Bytes/s",
           " Flow Packets/s",
           " Flow IAT Mean",
           " Flow IAT Std",
           " Flow IAT Max",
           " Flow IAT Min", 
           "Fwd IAT Total",
           " Fwd IAT Mean",
           " Fwd IAT Std",
           " Fwd IAT Max", 
           " Fwd IAT Min",
           "Bwd IAT Total", 
           " Bwd IAT Mean",
           " Bwd IAT Std", 
           " Bwd IAT Max", 
           " Bwd IAT Min", 
           " Fwd Header Length",
           " Bwd Header Length",
           "Fwd Packets/s",
           " Bwd Packets/s", 
           " Min Packet Length",
           " Max Packet Length",
           " Packet Length Mean",
           " Packet Length Std",
           " Packet Length Variance",
           " Avg Bwd Segment Size",
           " Average Packet Size",
           " Avg Fwd Segment Size",
           " Fwd Header Length",
           "Subflow Fwd Packets", 
           " Subflow Fwd Bytes", 
           " Subflow Bwd Packets",
           " Subflow Bwd Bytes",
           "Init_Win_bytes_forward", 
           " Init_Win_bytes_backward", 
           " act_data_pkt_fwd",
           " min_seg_size_forward",
           "Active Mean",
           " Active Std",
           " Active Max",
           " Active Min",
           "Idle Mean",
           " Idle Std",
           " Idle Max",
           " Idle Min"

 ]]

In [13]:
numer.shape     #  (445909,60)

(445909, 60)

In [14]:
#cater is the DataFrame that holds all of X's categorical variables
cater = X[["Fwd PSH Flags",
           " Bwd PSH Flags",  # 0
           " Fwd URG Flags",  # 0
           " Bwd URG Flags",  # 0
           "FIN Flag Count", 
           " SYN Flag Count",
           " RST Flag Count",
           " PSH Flag Count",
           " ACK Flag Count",
           " URG Flag Count",
           " CWE Flag Count",  # 0
           " ECE Flag Count",
           "Fwd Avg Bytes/Bulk",  # 0
           " Fwd Avg Packets/Bulk", # 0
           " Fwd Avg Bulk Rate",  # 0
           " Bwd Avg Bytes/Bulk",  # 0
           " Bwd Avg Packets/Bulk", # 0
           "Bwd Avg Bulk Rate"]]  # 0

In [15]:
cater.shape    # ((445909, 18))

(445909, 18)

In [16]:
listLabel = list(cater["Bwd Avg Bulk Rate"].drop_duplicates().values)
print('\n The types of all traffics are {}'.format(listLabel))


 The types of all traffics are [0]


So we can delete the redundant feature columns whose values are always zero

In [17]:
cater = cater.drop([" Bwd PSH Flags",  # 0
           " Fwd URG Flags",  # 0
           " Bwd URG Flags",
           " CWE Flag Count",
           "Fwd Avg Bytes/Bulk",  # 0
           " Fwd Avg Packets/Bulk", # 0
           " Fwd Avg Bulk Rate",  # 0
           " Bwd Avg Bytes/Bulk",  # 0
           " Bwd Avg Packets/Bulk", # 0
           "Bwd Avg Bulk Rate"], axis=1)

In [18]:
cater.shape     # (445909, 8)

(445909, 8)

In [19]:
numer.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Down/Up Ratio,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,88,640,7,4,440,358,0,220,0,62.857143,...,2,20,0.0,0.0,0,0,0.0,0.0,0,0
1,88,900,9,4,600,2944,0,300,0,66.666667,...,2,20,0.0,0.0,0,0,0.0,0.0,0,0
2,88,1205,7,4,2776,2830,0,1388,0,396.571429,...,2,20,0.0,0.0,0,0,0.0,0.0,0,0
3,88,511,7,4,452,370,0,226,0,64.571429,...,2,20,0.0,0.0,0,0,0.0,0.0,0,0
4,88,773,9,4,612,2944,0,306,0,68.000000,...,2,20,0.0,0.0,0,0,0.0,0.0,0,0


In [20]:
cater.head()

,Fwd PSH Flags,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,ECE Flag Count
0,0,0,0,0,1,0,0,0
1,0,0,0,0,1,0,0,0
2,0,0,0,0,1,0,0,0
3,0,0,0,0,1,0,0,0
4,0,0,0,0,1,0,0,0


Now that we have our separate numerical DataFrame, it's time to feature-scale it

In [21]:
#Initialize our scaler
#scaler = StandardScaler()

In [22]:
#Scale each column in numer: failed
#numer[' Destination Port'] = pd.DataFrame(scaler.fit_transform(numer[' Destination Port']))
# or numer = pd.DataFrame(scaler.fit_transform(numer))

In [23]:
numer = numer.rename(columns={"Flow Bytes/s":"Flow Bytes", " Flow Packets/s":"Flow Packets", "Fwd Packets/s":"Fwd Packets", " Bwd Packets/s":"Bwd Packets"})


In [25]:
# Failed
# numer["Flow Bytes"] = preprocessing.scale(numer["Flow Bytes"])

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [26]:
# Failed
# numer["Flow Packets"] = preprocessing.scale(numer["Flow Packets"])

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [27]:
numer["Fwd Packets"] = preprocessing.scale(numer["Fwd Packets"])
numer["Bwd Packets"] = preprocessing.scale(numer["Bwd Packets"])

In [33]:
numer[' Destination Port']=preprocessing.scale(numer[' Destination Port'])
numer[' Flow Duration']=preprocessing.scale(numer[' Flow Duration'])
numer[[' Total Fwd Packets',
       ' Total Backward Packets']]=preprocessing.scale(numer[[' Total Fwd Packets',' Total Backward Packets']])


H:\anacondaa\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning:

Data with input dtype int64 was converted to float64 by the scale function.



In [34]:
numer[[ "Total Length of Fwd Packets",
           " Total Length of Bwd Packets",
           " Down/Up Ratio",
           " Fwd Packet Length Max",
           " Fwd Packet Length Min",
           " Fwd Packet Length Mean",
           " Fwd Packet Length Std",
           "Bwd Packet Length Max",
           " Bwd Packet Length Min",
           " Bwd Packet Length Mean",
           ]]=preprocessing.scale(numer[[ "Total Length of Fwd Packets",
           " Total Length of Bwd Packets",
           " Down/Up Ratio",
           " Fwd Packet Length Max",
           " Fwd Packet Length Min",
           " Fwd Packet Length Mean",
           " Fwd Packet Length Std",
           "Bwd Packet Length Max",
           " Bwd Packet Length Min",
           " Bwd Packet Length Mean",
           ]])

In [35]:
numer[[" Bwd Packet Length Std"]]=preprocessing.scale(numer[[" Bwd Packet Length Std"]])
numer[" Flow IAT Mean"] = preprocessing.scale(numer[" Flow IAT Mean"])
numer[" Flow IAT Std"] = preprocessing.scale(numer[" Flow IAT Std"])
numer[" Flow IAT Max"] = preprocessing.scale(numer[" Flow IAT Max"])
numer[" Flow IAT Min"] = preprocessing.scale(numer[" Flow IAT Min"])
numer["Fwd IAT Total"] = preprocessing.scale(numer["Fwd IAT Total"])
numer[" Fwd IAT Mean"] = preprocessing.scale(numer[" Fwd IAT Mean"])
numer[" Fwd IAT Std"] = preprocessing.scale(numer[" Fwd IAT Std"])
numer[" Fwd IAT Max"] = preprocessing.scale(numer[" Fwd IAT Max"])
numer[" Fwd IAT Min"] = preprocessing.scale(numer[" Fwd IAT Min"])

H:\anacondaa\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning:

Data with input dtype int64 was converted to float64 by the scale function.



In [36]:
numer["Bwd IAT Total"] = preprocessing.scale(numer["Bwd IAT Total"])
numer[" Bwd IAT Mean"] = preprocessing.scale(numer[" Bwd IAT Mean"])
numer[" Bwd IAT Std"] = preprocessing.scale(numer[" Bwd IAT Std"])
numer[" Bwd IAT Max"] = preprocessing.scale(numer[" Bwd IAT Max"])
numer[" Bwd IAT Min"] = preprocessing.scale(numer[" Bwd IAT Min"])
numer[" Fwd Header Length"] = preprocessing.scale(numer[" Fwd Header Length"])
numer[" Bwd Header Length"] = preprocessing.scale(numer[" Bwd Header Length"])
numer[" Min Packet Length"] = preprocessing.scale(numer[" Min Packet Length"])
numer[" Max Packet Length"] = preprocessing.scale(numer[" Max Packet Length"])
numer[" Packet Length Mean"] = preprocessing.scale(numer[" Packet Length Mean"])
numer[" Packet Length Std"] = preprocessing.scale(numer[" Packet Length Std"])
numer[" Packet Length Variance"] = preprocessing.scale(numer[" Packet Length Variance"])  


H:\anacondaa\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning:

Data with input dtype int64 was converted to float64 by the scale function.



In [37]:
numer[" Avg Bwd Segment Size"] = preprocessing.scale(numer[" Avg Bwd Segment Size"])
numer[" Average Packet Size"] = preprocessing.scale(numer[" Average Packet Size"])
numer[" Avg Fwd Segment Size"] = preprocessing.scale(numer[" Avg Fwd Segment Size"])
numer[" Fwd Header Length"] = preprocessing.scale(numer[" Fwd Header Length"])
numer["Subflow Fwd Packets"] = preprocessing.scale(numer["Subflow Fwd Packets"])
numer[" Subflow Fwd Bytes"] = preprocessing.scale(numer[" Subflow Fwd Bytes"])
numer[" Subflow Bwd Packets"] = preprocessing.scale(numer[" Subflow Bwd Packets"])
numer[" Subflow Bwd Bytes"] = preprocessing.scale(numer[" Subflow Bwd Bytes"])
numer["Init_Win_bytes_forward"] = preprocessing.scale(numer["Init_Win_bytes_forward"])
numer[" Init_Win_bytes_backward"] = preprocessing.scale(numer[" Init_Win_bytes_backward"])
numer[" act_data_pkt_fwd"] = preprocessing.scale(numer[" act_data_pkt_fwd"])
numer[" min_seg_size_forward"] = preprocessing.scale(numer[" min_seg_size_forward"])
numer["Active Mean"] = preprocessing.scale(numer["Active Mean"])
numer[" Active Std"] = preprocessing.scale(numer[" Active Std"])
numer[" Active Max"] = preprocessing.scale(numer[" Active Max"])

H:\anacondaa\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning:

Data with input dtype int64 was converted to float64 by the scale function.



In [38]:
numer[" Active Min"] = preprocessing.scale(numer[" Active Min"])
numer["Idle Mean"] = preprocessing.scale(numer["Idle Mean"])
numer[" Idle Std"] = preprocessing.scale(numer[" Idle Std"])
numer[" Idle Max"] = preprocessing.scale(numer[" Idle Max"])
numer[" Idle Min"] = preprocessing.scale(numer[" Idle Min"])

H:\anacondaa\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning:

Data with input dtype int64 was converted to float64 by the scale function.



In [39]:
#Input contains NaN, infinity or a value too large for dtype('float64')
#numer['Flow Bytes'] = preprocessing.scale(numer['Flow Bytes']) 

In [40]:
#numer = numer.rename(columns={' Flow Packets/s':'Flow Packets'})
# Flow Bytes/s degismemis

Now we can re-merge our two DataFrames into a new, scaled X

In [41]:
X = pd.concat([numer, cater], axis=1, join='inner')

In [48]:
X.shape

(445909, 66)

In [43]:
X.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Down/Up Ratio,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,...,Idle Max,Idle Min,Fwd PSH Flags,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,ECE Flag Count
0,-0.45026,-0.364688,-0.005536,-0.008452,-0.015891,-0.008190,-1.286299,0.098748,-0.528528,0.126584,...,-0.263447,-0.25348,0,0,0,0,1,0,0,0
1,-0.45026,-0.364679,-0.003230,-0.008452,0.012294,-0.007205,-1.286299,0.268078,-0.528528,0.160238,...,-0.263447,-0.25348,0,0,0,0,1,0,0,0
2,-0.45026,-0.364668,-0.005536,-0.008452,0.395602,-0.007249,-1.286299,2.570966,-0.528528,3.074723,...,-0.263447,-0.25348,0,0,0,0,1,0,0,0
3,-0.45026,-0.364692,-0.005536,-0.008452,-0.013777,-0.008186,-1.286299,0.111447,-0.528528,0.141728,...,-0.263447,-0.25348,0,0,0,0,1,0,0,0
4,-0.45026,-0.364683,-0.003230,-0.008452,0.014407,-0.007205,-1.286299,0.280777,-0.528528,0.172018,...,-0.263447,-0.25348,0,0,0,0,1,0,0,0


In [46]:
X = X.drop(['Flow Bytes','Flow Packets'], axis=1)

In [47]:
X.to_csv("K:/exportedDFF.csv")